In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
!pip install pyvis
import pyvis
from pyvis.network import Network

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 47.7 MB/s eta 0:00:00


In [ ]:
#@title File Upload and Processing
#@markdown Upload your file and process it

from google.colab import files
import pandas as pd
import io

#@markdown ### Upload Settings
file_type = "Auto-detect" #@param ["Auto-detect", "CSV", "Excel", "Text", "JSON"]

#@markdown ---
#@markdown ### Data Upload Instructions
#@markdown * Run this cell (press ▶️ play button).
#@markdown * When the "Choose Files" button appears below, click it.
#@markdown * Select your file from your computer.
#@markdown * Your file will upload and automatically load.

def process_file():
  print("Waiting for file upload...")
  uploaded = files.upload()

  if not uploaded:
    print("No file was uploaded.")
    return None

  filename = next(iter(uploaded))
  file_content = uploaded[filename]
  print(f"File uploaded successfully: {filename}")

  # Auto-detect or use specified file type
  actual_file_type = file_type
  if file_type == "Auto-detect":
    if filename.endswith('.csv'):
      actual_file_type = "CSV"
    elif filename.endswith(('.xlsx', '.xls')):
      actual_file_type = "Excel"
    elif filename.endswith('.json'):
      actual_file_type = "JSON"
    else:
      actual_file_type = "Text"

  # Process based on file type
  try:
    if actual_file_type == "CSV":
      # Attempt to read with the first column as index
      try:
          df = pd.read_csv(io.BytesIO(file_content), index_col=0)
      except Exception:
          # Fallback if index_col=0 fails
          df = pd.read_csv(io.BytesIO(file_content))

      # Ensure all columns intended as numeric are indeed numeric, coercing errors to NaN
      # Assuming all columns except the index are meant to be numeric influence values
      for col in df.columns:
           df[col] = pd.to_numeric(df[col], errors='coerce')

      data = df
    elif actual_file_type == "Excel":
      # Attempt to read with the first column as index
      try:
          df = pd.read_excel(io.BytesIO(file_content), index_col=0)
      except Exception:
           # Fallback if index_col=0 fails
          df = pd.read_excel(io.BytesIO(file_content))

      # Ensure all columns intended as numeric are indeed numeric, coercing errors to NaN
      # Assuming all columns except the index are meant to be numeric influence values
      for col in df.columns:
           df[col] = pd.to_numeric(df[col], errors='coerce')
      data = df
    elif actual_file_type == "JSON":
      df = pd.read_json(io.BytesIO(file_content))
      data = df
    else:  # Text
      text_content = file_content.decode('utf-8')
      print(f"Text file loaded with {len(text_content)} characters")
      data = text_content


    return data

  except Exception as e:
    print(f"Error processing file: {str(e)}")
    return None

# Run the function when this cell is executed
df_influence = process_file()

Waiting for file upload...


Saving df_influence_v2.csv to df_influence_v2 (1).csv
File uploaded successfully: df_influence_v2 (1).csv


In [3]:
df_influence = pd.read_csv('df_influence_v2.csv', index_col=0)

performance = pd.DataFrame({
    'AUROC': [0.88],       # Good discriminative ability
    'f1': [0.80],          # Consistent with precision & recall
    'accuracy': [0.94],    # High accuracy but influenced by possible class imbalance
    'precision': [0.72],   # Indicates moderate false positives
    'recall': [0.90]       # Strong sensitivity, capturing most positives
})

In [4]:
def extract_top_pairs(df, top_n=10):
    """
    Extract the top feature-time influence pairs from the influence DataFrame.

    Parameters:
      df : pd.DataFrame
          DataFrame with rows and columns labeled as "Feature_X_t-Y" and cells
          containing influence numbers (NaN where influence is not defined).
      top_n : int
          Number of top pairs to extract.

    Returns:
      pd.DataFrame with the top feature-time influence pairs.
    """
    # Stack the DataFrame into a long format
    df_long = df.stack().reset_index()
    df_long.columns = ['Source', 'Target', 'Influence']

    # Convert the Influence column to numeric (coerce errors to NaN)
    df_long['Influence'] = pd.to_numeric(df_long['Influence'], errors='coerce')

    # Drop NaN entries (which include those originally NaN or non-numeric)
    df_long = df_long.dropna()

    # Compute absolute influence and sort descending
    df_long['AbsInfluence'] = df_long['Influence'].abs()
    df_top = df_long.sort_values(by='AbsInfluence', ascending=False).head(top_n)
    return df_top

# Example usage:
top_pairs = extract_top_pairs(df_influence, top_n=10)
print("Top 10 feature-time influence pairs:")
top_pairs


Top 10 feature-time influence pairs:


,Source,Target,Influence,AbsInfluence
34804,net_exp_I_t-5,n_claims_DR_t-6,0.004519,0.004519
31653,n_claims_I_t-4,CHF_t-5,-0.004210,0.004210
24633,CHF_t-3,CHF_t-4,0.003934,0.003934
29493,CHF_t-4,CHF_t-5,0.003324,0.003324
34840,net_exp_I_t-5,n_claims_DR_t-7,0.003019,0.003019
18477,CHF_t-2,CHF_t-3,0.003001,0.003001
34800,net_exp_I_t-5,Age_t-6,-0.002767,0.002767
34810,net_exp_I_t-5,net_exp_O_t-6,-0.002514,0.002514
4301,S5_t-0,S5_t-1,0.002487,0.002487
34806,net_exp_I_t-5,n_claims_O_t-6,0.002298,0.002298


In [5]:
def plot_influence_heatmap(df, figsize=(15, 12), title="Heatmap of Inter-Feature Influences"):
    """
    Plot a heatmap of the inter-feature influence DataFrame.

    Parameters:
      df : pd.DataFrame
          The influence matrix with NaNs where no valid influence exists
          (i.e., when the target time is not strictly after the source time).
      figsize : tuple
          Size of the figure (width, height).
      title : str
          Title for the plot.
    """
    plt.figure(figsize=figsize)

    # Create a mask to hide NaN values (the lower triangle or invalid cells)
    mask = df.isna()

    # Draw a heatmap with a diverging colormap, centered at zero
    sns.heatmap(df, mask=mask, cmap='RdBu', center=0, annot=False)

    plt.title(title)
    plt.xticks(rotation=90)  # rotate column labels
    plt.yticks(rotation=0)   # keep row labels horizontal
    plt.tight_layout()
    plt.show()

# Example usage:
# plot_influence_heatmap(df_influence)


In [8]:
def build_network_from_influence(df, threshold=0.001):
    """
    Build a directed network graph from the influence DataFrame.

    Parameters:
      df : pd.DataFrame
          A DataFrame where rows and columns are labeled with feature-time strings,
          and cells contain influence values (NaN where t2 <= t1 or no valid influence).
      threshold : float
          Minimum absolute influence to consider for adding an edge.

    Returns:
      nx.DiGraph
          A directed graph where each node is a feature-time label.
          An edge (u -> v) is added if the absolute influence from u to v
          is above the given threshold.
    """
    G = nx.DiGraph()

    # Convert the DataFrame to a long format: Source, Target, Influence
    df_long = df.stack().reset_index()
    df_long.columns = ["Source", "Target", "Influence"]

    # Drop rows with NaN influences (where t2 <= t1)
    df_long.dropna(subset=["Influence"], inplace=True)

    # Filter by threshold
    df_long = df_long[df_long["Influence"].abs() >= threshold]

    # Add edges to the graph
    for _, row in df_long.iterrows():
        source = row["Source"]
        target = row["Target"]
        influence = row["Influence"]
        G.add_node(source)
        G.add_node(target)
        G.add_edge(source, target, weight=influence)
        # print(f"Added edge from {source} to {target} with weight {influence}")

    return G

G = build_network_from_influence(df_influence, threshold=0.001)


In [9]:
def plot_interactive_network(G, notebook=True, output_file="interactive_network.html"):
    """
    Build and display an interactive network visualization using pyvis.
    """
    # Create a PyVis network object
    net = Network(height="750px", width="100%", notebook=notebook, directed=True, cdn_resources="in_line")
    net.force_atlas_2based(gravity=-50, central_gravity=0.01, spring_length=100, spring_strength=0.08)

    # Add nodes first
    for node in G.nodes():
        net.add_node(node, label=node)

    # Add edges with explicit weight transfer
    for u, v, data in G.edges(data=True):
        # Check if 'weight' exists, otherwise look for 'width'
        if 'weight' in data:
            weight = data['weight']
        elif 'width' in data:
            weight = data['width']
        else:
            weight = 1.0  # Default

        # Calculate edge width for visual display
        edge_width = abs(weight) * 3  # Scale for visibility

        # Set color based on weight
        edge_color = "red" if weight < 0 else "blue"

        # Add the edge with all attributes explicitly set
        net.add_edge(
            u, v,
            value=edge_width,          # Visual width
            width=edge_width,          # Alternative width parameter
            title=f"Value: {weight}",  # Tooltip text
            color=edge_color,          # Edge color
            physics=True,              # Enable physics
            smooth=True                # Smooth curves
        )

    # Add buttons for adjusting physics settings
    net.show_buttons(filter_=['physics'])

    # Save and show the graph
    net.show(output_file)

# Example usage:
# Assuming you already have a NetworkX DiGraph 'G' with a 'weight' attribute on edges:
# Corrected output_file path to save in the current directory
plot_interactive_network(G, notebook=True, output_file="interactive_network.html")

from IPython.display import HTML, display
# Corrected display path to look in the current directory
display(HTML('interactive_network.html'))

interactive_network.html


# LLM Enhancement

In [ ]:
import json

def extract_graph_info(G):
    """
    Extract all information from the NetworkX graph G and return it as a JSON string.
    The returned JSON includes:
      - A list of all nodes.
      - A list of all edges with their attributes (e.g., weight).
      - Summary statistics such as the number of nodes and edges.
    """
    # Extract nodes (feature-time labels)
    nodes = list(G.nodes())

    # Extract edges along with edge attributes
    edges = []
    for source, target, data in G.edges(data=True):
        edges.append({
            "source": source,
            "target": target,
            "weight": data.get("weight")
        })

    graph_info = {
        "number_of_nodes": G.number_of_nodes(),
        "number_of_edges": G.number_of_edges(),
        "nodes": nodes,
        "edges": edges
    }

    # Convert the dictionary to a formatted JSON string
    return json.dumps(graph_info, indent=2)

# Assuming G is your networkx DiGraph from the influence DataFrame:
graph_info = extract_graph_info(G)

# Save the graph info as a JSON file
with open("graph_info.json", "w") as f:
    json.dump(graph_info, f, indent=2)

# print(graph_info)

In [ ]:
import openai

class OpenAIAPIClient:
    """
    A simple client for calling the OpenAI API.

    The client takes in JSON information describing time-feature cross relationships
    (with top ranked relationships), and asks the language model to summarize and
    share several key observations or patterns.
    """

    def __init__(self, api_key: str, model: str = "gpt-4o", temperature: float = 0.2):
        """
        Initialize the OpenAIAPIClient.

        Parameters:
          api_key : str
              Your OpenAI API key.
          model : str
              The OpenAI model to use (default: "gpt-3.5-turbo").
          temperature : float
              Sampling temperature for the API call (default: 0.7).
        """
        self.api_key = api_key
        openai.api_key = self.api_key
        self.model = model
        self.temperature = temperature

    def generate_analysis(self, json_info: str) -> str:
        """
        Summarize the time-feature influence relationships from the provided JSON info.

        Parameters:
          json_info : str
              A string representing JSON data with the top ranked influence relationships.

        Returns:
          str: The summary generated by the LLM.
        """
        # System message to guide the model's response.

        system_message = (
            "This is the time-feature cross relationship we gained from previous deep learning modeling. "
            "The JSON contains the top ranked relationships. Summarize it and share several key observations or patterns."
        )

        # Create the conversation messages.
        messages = [
            {"role": "system", "content": system_message},
            {"role": "user", "content": f"Here is the JSON info:\n\n{json_info}"}
        ]

        # Call the OpenAI API.
        try:
            client = openai.OpenAI()
            completion = client.chat.completions.create(
                model=self.model,
                messages=messages,
                temperature=self.temperature
            )
            response = completion.choices[0].message.content

            return response, completion
        except Exception as e:
            print("Error calling OpenAI API:", e)
            return None




In [ ]:
from google.colab import userdata
import os

# Replace with your OpenAI API key.
os.environ["OPENAI_API_KEY"] = os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')



# Create an instance of the client.
chat = OpenAIAPIClient(api_key=os.getenv("OPENAI_API_KEY"))

# Load your JSON file (assumes you saved the graph info as a JSON file).
with open("graph_info.json", "r") as f:
    graph_content = f.read()

# Get the summary from the model.
summary, completion = chat.generate_analysis(graph_content)

# Print the summary.
print("Summary of Time-Feature Relationships:")
print(summary)

Summary of Time-Feature Relationships:
The JSON data represents a network of relationships between various time-lagged features, with nodes representing the features at different time steps and edges representing the relationships between these features. Here are some key observations and patterns from the data:

1. **Central Role of CHF (Congestive Heart Failure):**
   - The feature "CHF" at various time lags (t-0 to t-6) is highly interconnected with itself and other features, indicating its central role in the network. The strongest relationships are observed between consecutive time lags of CHF, such as CHF_t-3 and CHF_t-4 with a weight of 0.003934, and CHF_t-4 and CHF_t-5 with a weight of 0.003324.

2. **Significant Influence of S5:**
   - The feature "S5" also shows strong interconnections, particularly between its consecutive time lags. For example, S5_t-0 and S5_t-1 have a weight of 0.002487, and S5_t-3 and S5_t-4 have a weight of 0.002286. This suggests that S5 is another infl

# Dashboard

In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML
import markdown
import pandas as pd
import urllib.parse
import base64

# Inject custom CSS (same as before, for cloud-like tabs)
custom_css = """
<style>
/* Style all tabs with a cloud-like rounded appearance */
.widget-tab .p-TabBar-tab {
    border-radius: 20px;
    background-color: #f0f8ff; /* light, airy blue */
    margin: 4px;
    padding: 5px 10px;
    border: 1px solid #ccc;
    box-shadow: 2px 2px 5px rgba(0,0,0,0.1);
    transition: all 0.3s ease;
}

/* For the current (selected) tab, remove ALL colored borders */
.widget-tab .p-TabBar-tab.p-mod-current {
    background-color: #f0f8ff !important;
    border: 1px solid #ccc !important;
    font-weight: bold;
    border-top-color: #ccc !important;
    border-bottom-color: #ccc !important;
    border-left-color: #ccc !important;
    border-right-color: #ccc !important;
}

.widget-tab .p-TabBar-tab.p-mod-current::before,
.widget-tab .p-TabBar-tab.p-mod-current::after {
    border-top: none !important;
    background: none !important;
}
</style>
"""
display(HTML(custom_css))

def create_interactive_dashboard(summary_text, total_tokens, prompt_tokens, completion_tokens, model_name,
                                 dashboard_title="Time-Feature Influence Dashboard",
                                 model_performance_content="Model performance metrics will be displayed here.",
                                 key_timestamps_content="Key timestamps analysis will be displayed here.",
                                 key_features_content="Key features analysis will be displayed here.",
                                 key_timestamps_images="./content/temporal_importance.png",
                                 key_features_images=["./content/feature_importance.png", "./content/feature_at_time_importance.png"],
                                 cross_analysis_file="./content/interactive_network.html"):
    """
    Creates an interactive dashboard with a title and six tabs:
      - "Model Performance": Shows model performance metrics (DataFrame or Markdown).
      - "Key Timestamps": Shows key timestamps analysis with optional images.
      - "Key Features": Shows key features analysis with optional images.
      - "Cross Analysis": Embeds an interactive network visualization.
      - "LLM Summary": Shows the summary text (converted from Markdown to HTML).
      - "Token Usage": Shows a token usage table.

    All tabs use the same green-framed container style.

    Parameters:
    - key_timestamps_images: List of image paths for Key Timestamps tab
    - key_features_images: List of image paths for Key Features tab
    """
    def create_styled_content(title, content, image_paths=None, n_figs=None):
        """
        Create styled content with optional images.

        Parameters:
        - title: Title of the content
        - content: Markdown or HTML content
        - image_paths: A single image path or a list of image paths
        - n_figs: Number of figures to display (if None, will display all provided images)
        """
        # If content is not already HTML (a simple check), assume Markdown.
        if isinstance(content, str) and not content.lstrip().startswith("<"):
            content = markdown.markdown(content)

        # Add image(s) if provided
        image_html = ""

        # Handle image_paths parameter
        if image_paths is None:
            image_paths = []
        elif isinstance(image_paths, str):
            image_paths = [image_paths]  # Convert single path to list

        # Limit number of images if n_figs is specified
        if n_figs is not None and n_figs > 0:
            image_paths = image_paths[:n_figs]

        # If we have images
        if image_paths:
            try:
                # Determine number of images to be displayed
                num_images = len(image_paths)

                # Start flex container
                image_html = """
                <div style="display: flex; justify-content: space-around; flex-wrap: wrap; margin-bottom: 20px;">
                """

                # Create HTML for each image
                for i, img_path in enumerate(image_paths):
                    # Read image file and convert to base64
                    with open(img_path, "rb") as img_file:
                        img_data = base64.b64encode(img_file.read()).decode('utf-8')

                    # Determine image type
                    img_type = img_path.split('.')[-1].lower()
                    if img_type not in ['png', 'jpg', 'jpeg', 'gif']:
                        img_type = 'png'

                    # Calculate flex-basis based on number of images
                    # If single image: 90%, if two: 45%, if three or more: ~30%
                    flex_basis = "90%" if num_images == 1 else "45%" if num_images == 2 else "30%"

                    # Add image div
                    image_html += f"""
                    <div style="flex: 1; min-width: {flex_basis}; text-align: center; margin: 0 10px 10px 10px;">
                        <img src="data:image/{img_type};base64,{img_data}"
                             style="max-width: 100%; height: auto; max-height: 400px; border-radius: 5px; border: 1px solid #ddd;"
                             alt="{title} visualization {i+1}">
                    </div>
                    """

                # Close flex container
                image_html += """
                </div>
                """

            except Exception as e:
                image_html = f"<div style='color: red; margin-bottom: 10px;'>Error loading images: {e}</div>"

        return f"""
        <div style="border: 2px solid green; border-radius: 5px; padding: 15px; background-color: #e8f5e9;">
          <h2 style="color: darkgreen; margin-bottom: 10px;">{title}</h2>
          {image_html}
          {content}
        </div>
        """

    # Create LLM Summary tab.
    summary_html_content = markdown.markdown(summary_text)
    summary_html = create_styled_content("LLM Summary", summary_html_content)
    summary_widget = widgets.HTML(value=summary_html)

    # Create Token Usage tab.
    token_html = f"""
    <div style="border: 2px solid green; border-radius: 5px; padding: 15px; background-color: #e8f5e9;">
      <h2 style="color: darkgreen; margin-bottom: 10px;">Token Usage [Model: {model_name}]</h2>
      <table style="border-collapse: collapse; width: 60%; margin: auto;">
        <tr style="background-color: #f2f2f2;">
          <th style="border: 1px solid #dddddd; padding: 8px; text-align: center;">Token Type</th>
          <th style="border: 1px solid #dddddd; padding: 8px; text-align: center;">Count</th>
        </tr>
        <tr>
          <td style="border: 1px solid #dddddd; padding: 8px;">Total Tokens</td>
          <td style="border: 1px solid #dddddd; padding: 8px;">{total_tokens}</td>
        </tr>
        <tr>
          <td style="border: 1px solid #dddddd; padding: 8px;">Prompt Tokens</td>
          <td style="border: 1px solid #dddddd; padding: 8px;">{prompt_tokens}</td>
        </tr>
        <tr>
          <td style="border: 1px solid #dddddd; padding: 8px;">Completion Tokens</td>
          <td style="border: 1px solid #dddddd; padding: 8px;">{completion_tokens}</td>
        </tr>
      </table>
    </div>
    """
    token_widget = widgets.HTML(value=token_html)

    # Extra tabs.
    # Model Performance tab.
    if isinstance(model_performance_content, pd.DataFrame):
        # You could use a helper function to render DataFrame as HTML; here we assume a function render_performance_html
        mp_html = render_performance_html(model_performance_content)
    else:
        mp_html = create_styled_content("Model Performance", model_performance_content)
    mp_widget = widgets.HTML(value=mp_html)

    # Key Timestamps tab - now with image(s).
    kt_html = create_styled_content("Key Timestamps", key_timestamps_content,
                                   image_paths=key_timestamps_images,
                                   n_figs=1)  # Default to show just 1 image
    kt_widget = widgets.HTML(value=kt_html)

    # Key Features tab - now with multiple side-by-side images.
    kf_html = create_styled_content("Key Features", key_features_content,
                                   image_paths=key_features_images,
                                   n_figs=2)  # Default to show 2 images
    kf_widget = widgets.HTML(value=kf_html)

    # Cross Analysis tab.
    # Read the HTML file and embed it via a data URL in an iframe.
    try:
        with open(cross_analysis_file, "r", encoding="utf-8") as f:
            network_html = f.read()
        # URL-encode the HTML content.
        encoded_html = urllib.parse.quote(network_html)
        iframe_html = f'<iframe src="data:text/html;charset=utf-8,{encoded_html}" width="100%" height="750px" style="border: none;"></iframe>'
    except Exception as e:
        iframe_html = f"<p>Error loading interactive network: {e}</p>"
    ca_html = create_styled_content("Cross Analysis", iframe_html)
    ca_widget = widgets.HTML(value=ca_html)

    # Order: Model Performance, Key Timestamps, Key Features, Cross Analysis, LLM Summary, Token Usage.
    tab_widgets = [mp_widget, kt_widget, kf_widget, ca_widget, summary_widget, token_widget]
    tab_titles = ["Model Performance", "Key Timestamps", "Key Features", "Cross Analysis", "LLM Summary", "Token Usage"]

    tab = widgets.Tab(children=tab_widgets)
    for idx, title in enumerate(tab_titles):
        tab.set_title(idx, title)

    # Create a dashboard title widget.
    title_widget = widgets.HTML(
        value=f"<h1 style='text-align: center; color: darkblue;'>{dashboard_title}</h1>"
    )

    # Combine title and tab widget.
    dashboard = widgets.VBox([title_widget, tab])
    display(dashboard)

def render_performance_html(df):
    """
    Convert a one-row performance DataFrame into an HTML table wrapped in a green-framed container.
    """
    rows_html = ""
    for col in df.columns:
        value = df.iloc[0][col]
        rows_html += f"""
        <tr>
            <td style="border: 1px solid #dddddd; padding: 8px;">{col}</td>
            <td style="border: 1px solid #dddddd; padding: 8px;">{value}</td>
        </tr>
        """
    table_html = f"""
    <table style="border-collapse: collapse; width: 60%; margin: auto;">
      <tr style="background-color: #f2f2f2;">
        <th style="border: 1px solid #dddddd; padding: 8px; text-align: center;">Metric</th>
        <th style="border: 1px solid #dddddd; padding: 8px; text-align: center;">Value</th>
      </tr>
      {rows_html}
    </table>
    """
    html = f"""
    <div style="border: 2px solid green; border-radius: 5px; padding: 15px; background-color: #e8f5e9;">
      <h2 style="color: darkgreen; margin-bottom: 10px;">Model Performance</h2>
      {table_html}
    </div>
    """
    return html


# Example usage:

total_tokens = 4309
prompt_tokens = 3826
completion_tokens = 483
model_name = "gpt-4o"


# Define content for the extra tabs.
key_timestamps = """
**Critical Time Points:**
* 2024-01-15: Major spike in activity.
* 2024-02-03: System downtime affected metrics.
* 2024-03-22: New feature release.
"""
key_features = """
**Feature Importance Rankings:**
1. User login frequency (0.85)
2. Session duration (0.72)
3. Purchase history (0.67)
4. Geographic location (0.54)
"""

# Provide image URLs (or local paths accessible via a web server) for key timestamps and key features.
# For example, if the images are hosted somewhere accessible.




create_interactive_dashboard(
    summary_text,
    total_tokens,
    prompt_tokens,
    completion_tokens,
    model_name,
    dashboard_title="Time-Feature Influence Dashboard",
    model_performance_content=performance,  # Passing a DataFrame
    key_timestamps_content=key_timestamps,
    key_features_content=key_features
)
